In [1]:
import json
import networkx as nx
from pyvis.network import Network

# Your JSON Data
data = {
    "metadata": {"engine": "Gemini-Forensics-2026"},
    "steps": [
        {"hop": 1, "txid": "4bbd...", "sink": {"address": "Unknown/OP_RETURN", "value": 0}, "router": {"address": "bitcoincash:qzcal...", "value": 0.00106478}},
        {"hop": 2, "txid": "0c81...", "sink": {"address": "bitcoincash:qzcal...", "value": 1e-05}, "router": {"address": "bitcoincash:qzcal...", "value": 0.00207538}},
        {"hop": 3, "txid": "5c72...", "sink": {"address": "bitcoincash:qq7sg...", "value": 0.001448}, "router": {"address": "bitcoincash:qzcal...", "value": 0.0021}}
        # ... add more steps here
    ]
}

def generate_forensic_map(json_data):
    # Initialize interactive network
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", directed=True)
    
    for step in json_data['steps']:
        router = step['router']['address']
        sink = step['sink']['address']
        txid = step['txid']
        value = step['sink']['value']
        hop = step['hop']

        # Add Router Node (Source)
        net.add_node(router, label=f"Addr: {router[:10]}...", title=router, color="#ff4b4b")
        
        # Add Sink Node (Destination)
        net.add_node(sink, label=f"Addr: {sink[:10]}...", title=sink, color="#1f78ff")
        
        # Add Edge (The Transaction Trail)
        edge_label = f"Hop {hop}: {value} BCH"
        net.add_edge(router, sink, label=edge_label, title=f"TXID: {txid}")

    # Set physics for a "Fan-Out" aesthetic
    net.barnes_hut()
    net.show("money_trail.html", notebook=False)
    print("Forensic map generated: open 'money_trail.html' in your browser.")

# Run the script
generate_forensic_map(data)

money_trail.html
Forensic map generated: open 'money_trail.html' in your browser.


In [5]:
import networkx as nx
from pyvis.network import Network

# 1. Initialize NetworkX Graph
G = nx.Graph()

# 2. Example: Loading your JSON structure
# (Assuming a root node 'A' and multiple children 'B', 'C', etc.)
edges = [('Root', 'Child1'), ('Root', 'Child2'), ('Root', 'Child3'), 
         ('Child1', 'Sub1'), ('Child1', 'Sub2'), ('Root', 'Child4')]
G.add_edges_from(edges)

# 3. Calculate "Shell" or "Radial" Layout
# This assigns (x, y) coordinates in a clean circular fan-out
pos = nx.shell_layout(G) 

# 4. Transfer to Pyvis
nt = Network(height='600px', width='100%', notebook=True, bgcolor='#FFFFFF', font_color='red')

for node, coords in pos.items():
    # We multiply coords by a factor (e.g., 400) to give the nodes breathing room
    nt.add_node(node, x=coords[0] * 1000, y=coords[1] * 1000, physics=False)

for edge in G.edges():
    nt.add_edge(edge[0], edge[1])

# 5. Physics Tuning
# Disabling 'solver' stops the nodes from bouncing around after loading
nt.toggle_physics(False)
nt.show('nx1.html')

nx1.html


In [7]:
import json
import networkx as nx
from pyvis.network import Network

# 1. Load your data
with open('bch_map.json', 'r') as f:
    data = json.load(f)

# 2. Build the NetworkX Graph
G = nx.Graph()

# Mapping logic (Assumes your JSON has 'nodes' and 'edges' or a similar structure)
# Adjust these keys if your JSON uses different names like 'source'/'target'
for item in data.get('edges', []):
    G.add_edge(item['from'], item['to'])

# 3. Calculate the "Perfect" Radial Layout
# shell_layout creates the circular fan-out pattern
pos = nx.shell_layout(G)

# 4. Initialize Pyvis with your styling
nt = Network(height='800px', width='100%', notebook=True, bgcolor='black', font_color='red')

# 5. Add nodes with big fonts and fixed positions
for node, coords in pos.items():
    nt.add_node(
        node, 
        label=str(node),
        x=coords[0] * 2000,  # Huge multiplier to give labels room
        y=coords[1] * 2000, 
        size=25,             # Makes the actual node dot bigger
        font={'size': 40},   # BIG red font
        physics=False        # Essential: Keeps the "perfect" fan-out from moving
    )

for edge in G.edges():
    nt.add_edge(edge[0], edge[1], color='gray', width=2)

# 6. Save and view
nt.show('perfect_fanout.html')

perfect_fanout.html


In [11]:
import json
import networkx as nx
from pyvis.network import Network

# 1. Load your specific BCH data
# Make sure your file is named 'bch_map.json' or update the name below
with open('bch_map.json', 'r') as f:
    data = json.load(f)

# 2. Build the NetworkX Graph
G = nx.Graph()

# Mapping logic adjusted for your "steps" -> "router" & "sink" structure
for step in data.get('steps', []):
    source = step['router']['address']
    target = step['sink']['address']
    
    # We add edges. NetworkX automatically handles duplicate nodes.
    G.add_edge(source, target)

# 3. Calculate the "Perfect" Radial Layout
# Shell layout creates the clean circular fan-out forensics pattern
pos = nx.shell_layout(G)

# 4. Initialize Pyvis with your high-contrast forensic styling
# Note: set notebook=False if running as a standard .py script
nt = Network(height='800px', width='100%', notebook=False, bgcolor='white', font_color='red')

# 5. Add nodes with BIG fonts and fixed positions
for node, coords in pos.items():
    # Forensic touch: Trim long BCH addresses for the label, keep full in title (hover)
    display_label = f"{node[:10]}...{node[-5:]}" 
    
    nt.add_node(
        node, 
        label=display_label, 
        title=node,          # Shows full address on hover
        x=coords[0] * 3000,  # Increased to 3000 for even more breathing room
        y=coords[1] * 3000, 
        size=30,             # Larger node dots
        font={'size': 45, 'color': 'red', 'face': 'monospace'}, 
        physics=False        # Locks the fan-out in place
    )

# 6. Add the connection lines (the money trail)
for edge in G.edges():
    nt.add_edge(edge[0], edge[1], color='gray', width=3)

# 7. Save and generate the forensic report
nt.write_html('perfect_fanout1.html')
print("Forensic map generated successfully. Open 'perfect_fanout.html' in your browser.")

Forensic map generated successfully. Open 'perfect_fanout.html' in your browser.


In [17]:
import json
import networkx as nx
from pyvis.network import Network

# 1. Load your BCH Forensic JSON
try:
    with open('bch_map.json', 'r') as f:
        data = json.load(f)
except FileNotFoundError:
    print("Error: 'bch_map.json' not found. Ensure the file is in the same folder.")
    data = {"steps": []}

# 2. Build the Directed Graph (DiGraph shows money flow direction)
G = nx.DiGraph()

for step in data.get('steps', []):
    source = step['router']['address']
    target = step['sink']['address']
    val = step['sink']['value']
    tx = step['txid']
    hop = step['hop']
    
    # Store forensic metadata in the edge
    G.add_edge(source, target, title=f"Hop: {hop}\nValue: {val}\nTXID: {tx}")

# 3. Radial "Shell" Layout
pos = nx.shell_layout(G)

# 4. Initialize Pyvis with High-Contrast Styling
nt = Network(height='800px', width='100%', notebook=False, bgcolor='black', font_color='red')

# 5. Add nodes with fixed radial positions
for node, coords in pos.items():
    # Forensic Label: Show first/last 5 chars of the BCH address
    display_label = f"{node[:7]}...{node[-5:]}" if ":" in node else node
    
    nt.add_node(
        node, 
        label=display_label, 
        title=node,          # Full address on hover
        x=coords[0] * 2500,  # Spread out nodes
        y=coords[1] * 2500, 
        size=30, 
        font={'size': 40, 'color': 'red'}, 
        physics=False
    )

# 6. Add directed edges
for u, v, attr in G.edges(data=True):
    nt.add_edge(u, v, title=attr['title'], color='gray', width=2, arrows='to')

# 7. Use write_html to bypass the 'NoneType' rendering error
nt.write_html('perfect_fanout.html')
print("File 'perfect_fanout.html' has been created successfully.")

File 'perfect_fanout.html' has been created successfully.


In [21]:
import json
import networkx as nx
from pyvis.network import Network

# 1. Load Data
with open('bch_map.json', 'r') as f:
    data = json.load(f)

# 2. Initialize Network with "Repulsion" physics for auto-fit
nt = Network(height='90vh', width='100%', bgcolor='black', font_color='white', directed=True)

# 3. Process Steps
for step in data.get('steps', []):
    src = step['router']['address']
    snk = step['sink']['address']
    val = float(step['sink']['value'])
    tx = step['txid']
    
    # Label Trimming for Visibility
    src_label = f"{src[:6]}...{src[-4:]}"
    snk_label = f"{snk[:6]}...{snk[-4:]}"

    # Add Source Node (Bigger size for routers)
    nt.add_node(src, label=src_label, title=src, color='#FF0000', size=40)
    
    # Add Sink Node
    nt.add_node(snk, label=snk_label, title=snk, color='#00FFFF', size=25)
    
    # Add HEAVY Flow Lines (Width scales with value)
    # We use (val * 1000) + 2 so even small txs are visible, but big ones are HEAVY
    line_width = (val * 5000) + 3 
    nt.add_edge(src, snk, width=line_width, color='gray', title=f"Value: {val} BCH\nTXID: {tx}")

# 4. Forensic "Force-Directed" Physics (The Auto-Fit Secret)
nt.set_options("""
var options = {
  "nodes": {
    "font": { "size": 32, "face": "courier", "strokeWidth": 2, "strokeColor": "#000000" }
  },
  "edges": {
    "arrows": { "to": { "enabled": true, "scaleFactor": 1.5 } },
    "color": { "inherit": true },
    "smooth": { "type": "curvedCW", "roundness": 0.2 }
  },
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -100,
      "centralGravity": 0.01,
      "springLength": 200,
      "springConstant": 0.08
    },
    "maxVelocity": 50,
    "solver": "forceAtlas2Based",
    "timestep": 0.35,
    "stabilization": { "iterations": 150 }
  }
}
""")

# 5. Output
nt.write_html('forensic_impact_map.html')
print("Heavy Flow Map Generated: forensic_impact_map.html")

Heavy Flow Map Generated: forensic_impact_map.html
